In [31]:
# load data func

from __future__ import print_function
import glob, os, numpy as np
from keras.preprocessing import image as kImage

def getData(dataset_dir):
    X_list= sorted(glob.glob(os.path.join(dataset_dir, 'x','*.jpg')))
    Y_list = sorted(glob.glob(os.path.join(dataset_dir, 'y' ,'*.png')))
    
    X= []
    Y= []
    for i in range(len(X_list)):
        # Load input image
        x = kImage.load_img(X_list[i])
        x = kImage.img_to_array(x)
        X.append(x)
        
        # Load ground-truth label and encode to value 0 and 1
        x = kImage.load_img(Y_list[i])
        x = kImage.img_to_array(x)
        #x /= 255.0
        #x = np.floor(x)
        Y.append(x)
        
    X = np.asarray(X)
    Y = np.asarray(Y)
    
    # Shuffle the training data
    idx = list(range(X.shape[0]))
    np.random.shuffle(idx)
    X = X[idx]
    Y = Y[idx]
    
    return X, Y

In [32]:
import keras
from keras.models import Model
from keras.layers import Deconvolution2D as Deconv2D, Input
import keras.backend as K
import tensorflow as tf
from keras import regularizers

def initModel():
    ### Encoder
    net_input = Input(shape=(240,320,3))
                
    vgg16 = keras.applications.vgg16.VGG16(include_top=False, weights='imagenet', input_tensor=net_input)

    # vgg16 = keras.applications.Xception(include_top=False, weights='imagenet', input_tensor=net_input)
    # for layer in vgg16.layers[:130]:
      # layer.trainable = False
    # vgg16 = keras.applications.vgg16.VGG16(include_top=False, weights=None, input_tensor=net_input)
    # for layer in vgg16.layers[:17]:
    #  layer.trainable = True
    for layer in vgg16.layers[:17]:
      layer.trainable = False
    
    x = vgg16.layers[-2].output # 2nd layer from the last, block5_conv3
    
    ### Decoder
    x = Deconv2D(256, (3,3), strides=(2,2), activation='relu', padding='same')(x)
    x = Deconv2D(128, (3,3), strides=(2,2), activation='relu', padding='same')(x)
    x = Deconv2D(64, (3,3), strides=(2,2), activation='relu', padding='same')(x)
    x = Deconv2D(32, (3,3), strides=(2,2), activation='relu', padding='same')(x)
    x = Deconv2D(3, (1,1), activation='sigmoid', padding='same')(x)


    model = Model(inputs=vgg16.input, outputs=x)
    # Разные виды оптимизаторов
    model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.RMSprop(lr=5e-4), metrics=['accuracy'])
    # model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.SGD(lr=5e-4), metrics=['accuracy'])
    # model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adam(lr=5e-4), metrics=['accuracy'])
    # model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adadelta(lr=5e-4), metrics=['accuracy'])
    # model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adagrad(lr=5e-4), metrics=['accuracy'])
    # model.compile(loss=keras.losses.categorical_crossentropy, optimizer=keras.optimizers.Adamax(lr=5e-4), metrics=['accuracy'])
    return model

In [33]:
# dot_img_file = 'model.png'
# tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)
# len(model.layers)

In [ ]:
# load data
#dataset_path = os.path.join('CDnet2014', 'new_train')
dataset_path = os.path.join('drive', 'My Drive', 'my_net', 'train')
X, Y = getData(dataset_path)
#plt.imshow(Y[0].astype('uint8'))

# init the model
model = initModel()

early = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=1e-4, patience=10)
reduce = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5)


model.fit(X, Y, batch_size=1, epochs=100, verbose=2, validation_split=0.2, callbacks=[reduce, early], shuffle=True)
# model.count_params()
# model.summary()
# trainable_count = int(np.sum([K.count_params(p) for p in set(model.trainable_weights)]))
model.save('my_model.h5')

# dot_img_file = 'model.png'  
# tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)

In [ ]:
from keras import backend as K
import tensorflow as tf
import os

dataset_path = os.path.join('drive', 'My Drive', 'my_net', 'test')
X, Y = getData(dataset_path)
pred = model.predict(X, verbose=1, batch_size=1)

model.evaluate(X, Y)

#print(tf.Session().run(K.mean(K.equal(Y, K.round(pred)))))

In [ ]:
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (12.0, 9.0)

idx = 0 # image index in a range of [0,9]

img = np.empty(3, dtype=object)
img[0] = X[idx]
img[1] = Y[idx]
img[2] = np.around(pred[idx]*255)
np.around(pred[idx]*255).shape
title = ['входое изображение', 'ручная сегментация', 'программная сегментация']
for i in range(3):
  plt.subplot(1, 3, i+1)
  plt.imshow(img[i].astype('uint8'))
  plt.axis('off')
  plt.title(title[i])
plt.show()

# Сохранить результат сегментации:
# fig = plt.figure()
# plt.imshow(img[2].astype('uint8'))
# plt.axis('off')
# plt.show()
# fig.savefig('plot.png')

# Показать входное и выходное изображение после программной сегментации:
# fig = plt.figure()
# plt.rcParams['figure.figsize'] = (24.0, 18.0)
# plt.subplot(1, 2, 1)
# plt.imshow(img[1].astype('uint8'))
# plt.axis('off')
# plt.title(title[1])

# plt.subplot(1, 2, 2)
# plt.imshow(img[2].astype('uint8'))
# plt.axis('off')
# plt.title(title[2])
# fig.patch.set_facecolor('white')
# plt.show()
# fig.savefig('plot.png')